In [1]:
cd Deep\ active\ contours

[Errno 2] No such file or directory: 'Deep active contours'
/home/ahabis/3-Deep_active_contour/scores


In [103]:
import SimpleITK as sitk
from ripser import ripser
from persim import plot_diagrams
import matplotlib.pyplot as plt
from sklearn.metrics import balanced_accuracy_score, pairwise_distances
from sklearn.linear_model import LinearRegression
from radiomics import featureextractor
# extractor = featureextractor.RadiomicsFeatureExtractor('glcm.yaml')
import re
import numpy as np
import pandas as pd
from operator import itemgetter
from pyfeats import correlogram
# from config import *
from scipy.ndimage import distance_transform_edt

In [179]:
def process_score(x):
    x = re.sub(' +', ',',x)
    x = x.replace('nan','0')
    arr = np.array([1/(2**i) for i in range(5)])
    arr = arr/np.sum(arr)
    return np.dot(np.array(eval(x))[:-1],arr)
eps = 1e-10
THRESHOLD = 0.9693
scores = pd.read_csv('scores.csv', index_col=0)
scores['score'] = scores['score'].apply(process_score)
scores['pred'] = (scores['score']>THRESHOLD).astype(int)
scores['tp'] = scores['gt'] * scores['pred']
scores['fp'] = (1 - scores['gt']) * scores['pred']
scores['fn'] = scores['gt'] * (1 - scores['pred'])
scores['tn'] = (1 - scores['gt']) * (1 - scores['pred'])

nb_slides = len(np.unique(scores['slide']))
precisions = np.zeros((nb_slides,10))
recalls = np.zeros((nb_slides,10))
ious = np.zeros((nb_slides,10))
dices = np.zeros((nb_slides,10))
pqs = np.zeros((nb_slides,10))
f1s = np.zeros((nb_slides,10))

for i, slide in enumerate(np.unique(scores['slide'])):
    df_slide = scores[scores['slide'] == slide]
    for j, nb_anchor in enumerate(np.unique(df_slide['nb_image'])):
        df_anchor_slide = df_slide[df_slide['nb_image'] == nb_anchor]
        gt = np.array(df_anchor_slide['gt'])
        pred = np.array(df_anchor_slide['pred']).astype(int)
        dice = np.array(df_anchor_slide['DICE(%)']).astype(float)
        iou = np.array(df_anchor_slide['IOU(%)']).astype(float)
        iou_tp = np.sum(iou*pred*gt)/np.sum(pred*gt)
        dice_tp = np.sum(dice*pred*gt)/np.sum(pred*gt)
        tp = df_anchor_slide['tp']
        tn = df_anchor_slide['tn']
        fp = df_anchor_slide['fp']
        fn = df_anchor_slide['fn']
        precision =   np.sum(tp) / (np.sum(tp) + np.sum(fp) + eps) 
        recall =   np.sum(tp) / (np.sum(tp) + np.sum(fn) + eps) 
        f1 = (2*precision*recall)/(precision + recall + eps)
        precisions[i,j] = precision
        recalls[i,j] = recall
        dices[i,j] = dice_tp
        ious[i,j] = iou_tp
        pqs[i,j] = iou_tp * f1
        f1s[i,j] =  f1

final_f1 = np.mean(f1s)
final_dices = np.mean(dices)
final_recalls = np.mean(recalls)
final_precisions = np.mean(precisions)
final_pqs = np.mean(pqs)
final_ious = np.mean(ious)


In [188]:
recalls.shape
print('mean')
print(np.mean(recalls))
print(np.mean(precisions))
print(np.mean(f1s))
print(np.mean(dices))
print(np.mean(ious))
print(np.mean(pqs))

print('max')
print(np.mean(np.max(recalls,axis = -1)))
print(np.mean(np.max(precisions,axis = -1)))
print(np.mean(np.max(f1s,axis = -1)))
print(np.mean(np.max(dices,axis = -1)))
print(np.mean(np.max(ious,axis = -1)))
print(np.mean(np.max(pqs,axis = -1)))

print('min')

print(np.mean(np.min(recalls,axis = -1)))
print(np.mean(np.min(precisions,axis = -1)))
print(np.mean(np.min(f1s,axis = -1)))
print(np.mean(np.min(dices,axis = -1)))
print(np.mean(np.min(ious,axis = -1)))
print(np.mean(np.min(pqs,axis = -1)))

print('std')
print(np.mean(np.std(recalls,axis = -1)))
print(np.mean(np.std(precisions,axis = -1)))
print(np.mean(np.std(f1s,axis = -1)))
print(np.mean(np.std(dices,axis = -1)))
print(np.mean(np.std(ious,axis = -1)))
print(np.mean(np.std(pqs,axis = -1)))

mean
0.8800619974968596
0.6974886826504103
0.7608392334442536
82.86958424513841
71.99593853613902
54.947808519864424
max
0.9910368448305913
0.8572853072794174
0.8643035611987647
87.14322051401756
77.8166935023582
64.92233599507274
min
0.6288953367629216
0.5931447113916725
0.6430141352592887
75.6912007264213
62.39479337276703
43.580531063259166
std
0.1122924401838886
0.08416190483220418
0.06689582800860532
3.40501836483944
4.586544991067897
6.586031498248894
